## Lista de exercícios - 011

### Francisco Romes da Silva Filho, 409976

## Resolva o problema de classificação binária sobre o dataset "Pima Indians Diabetes Database"

Montando o Drive!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Instalando depedências

In [2]:
#XGBoost
!pip install xgboost

#LightGBM
!pip install lightgbm

#CatBoost
!pip install catboost



     |████████████████████████████████| 67.4 MB 27 kB/s 


In [3]:
!pip install pipelinehelper 

### Imports de bibliotecas

In [4]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from pipelinehelper import PipelineHelper
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

### Dataset

In [5]:
# Pima Indians Diabetes Database
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetes.csv')
print("Forma do dataset: ", data.shape[1], " features e ", data.shape[0], " amostras!")

Forma do dataset:  9  features e  768  amostras!


In [6]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## 1. Separe os dados em treino (80%) e teste (20%).


In [7]:
X = data.drop(['Outcome'], axis=1)
y = data['Outcome']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

## 2. Crie os modelos a seguir sobre o conjunto de treino e calcule F1-Score sobre os dados de treino e de teste usando:


#### 1. Random Forest

In [9]:
pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
    ])),
    ('classifier', PipelineHelper([
        ('rf', RandomForestClassifier()),
    ])),
])

params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate({
        'std__with_mean': [True, False],
        'std__with_std': [True, False],
        'max__copy': [True],  # just for displaying
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'rf__n_estimators': [100, 20],
    })
}


In [10]:
grid = GridSearchCV(pipe, params, scoring='f1', verbose=1, n_jobs=-1)

In [11]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.5s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        PipelineHelper(available_models={'max': MaxAbsScaler(copy=True),
                                                                         'std': StandardScaler(copy=True,
                                                                                               with_mean=True,
                                                                                               with_std=True)},
                                                       optional=False,
                                                       selected_model=None)),
                                       ('classifier',
                                        PipelineHelper(available_models={'rf': RandomForestClassifier(bootstrap=True,
                                                                                                      ccp_alpha

In [12]:
print(grid.best_params_)
print(grid.best_score_)

{'classifier__selected_model': ('rf', {'n_estimators': 100}), 'scaler__selected_model': ('std', {'with_mean': False, 'with_std': True})}
0.6194644861311527


In [13]:
model = grid
rf_pred = model.predict(X_test)

In [14]:
f1_score(y_test, rf_pred)

0.6734693877551021

### 2. Gradient Boosting do Scikit Learn


In [15]:
pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
    ])),
    ('classifier', PipelineHelper([
        ('gb', GradientBoostingClassifier()),
    ])),
])

params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate({
        'std__with_mean': [True, False],
        'std__with_std': [True, False],
        'max__copy': [True],  # just for displaying
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'gb__n_estimators': [100, 200, 300],
        'gb__loss': ['deviance', 'exponential'],
        'gb__max_depth': [2, 3, 4, 5],
    })
}


In [16]:
grid = GridSearchCV(pipe, params, scoring='f1', verbose=1, n_jobs=-1)

In [17]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done 514 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  2.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        PipelineHelper(available_models={'max': MaxAbsScaler(copy=True),
                                                                         'std': StandardScaler(copy=True,
                                                                                               with_mean=True,
                                                                                               with_std=True)},
                                                       optional=False,
                                                       selected_model=None)),
                                       ('classifier',
                                        PipelineHelper(available_models={'gb': GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                                          cr

In [18]:
print(grid.best_params_)
print(grid.best_score_)

{'classifier__selected_model': ('gb', {'loss': 'deviance', 'max_depth': 3, 'n_estimators': 100}), 'scaler__selected_model': ('std', {'with_mean': True, 'with_std': False})}
0.6281770778949163


In [19]:
model = grid
gb_pred = model.predict(X_test)

In [20]:
f1_score(y_test, gb_pred)

0.72

#### 3. XGBoost 

In [21]:
xgb = XGBClassifier(objective="binary:logistic")
param_grid = {
    "max_depth": [3, 4, 5, 7],
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.1, 0.01, 0.05],
}

In [22]:
grid_cv = GridSearchCV(xgb, param_grid, n_jobs=-1, cv=5, scoring="f1")


In [23]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': [3, 4, 5, 7],
                         'n_estimators': [100, 200, 300]},
       

In [24]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}
0.6307482655157075


In [25]:
xgb = grid_cv
xgb_pred = xgb.predict(X_test)

In [26]:
f1_score(y_test, xgb_pred)

0.6666666666666667

#### 4. LightGBM

In [36]:
lgb = LGBMClassifier()
param_grid = {'num_leaves':[20,40,60], 'max_depth':[-1, 5, 10],
             'learning_rate':[0.05, 0.1,0.2]}

In [37]:
grid_cv = GridSearchCV(lgb, param_grid, n_jobs=-1, cv=5, scoring="f1")


In [38]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth'

In [39]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'learning_rate': 0.05, 'max_depth': 10, 'num_leaves': 20}
0.6143723574076565


In [40]:
lgbm = grid_cv
lgbm_pred = lgbm.predict(X_test)

In [41]:
f1_score(y_test, lgbm_pred)

0.6666666666666666

#### 5. CatBoost

In [28]:
cbc = CatBoostClassifier()
param_grid = {'depth'         : [4, 5, 7],
              'learning_rate' : [0.01,0.02,0.03],
}

In [29]:
grid_cv = GridSearchCV(cbc, param_grid, n_jobs=-1, cv=5, scoring="f1")


In [30]:
grid_cv.fit(X_train, y_train)

0:	learn: 0.6815632	total: 51.5ms	remaining: 51.5s
1:	learn: 0.6722663	total: 53.8ms	remaining: 26.8s
2:	learn: 0.6621847	total: 56ms	remaining: 18.6s
3:	learn: 0.6529564	total: 58.1ms	remaining: 14.5s
4:	learn: 0.6423249	total: 60.3ms	remaining: 12s
5:	learn: 0.6342900	total: 62.4ms	remaining: 10.3s
6:	learn: 0.6248554	total: 64.5ms	remaining: 9.16s
7:	learn: 0.6172942	total: 66.7ms	remaining: 8.27s
8:	learn: 0.6092561	total: 68.9ms	remaining: 7.58s
9:	learn: 0.6013613	total: 71ms	remaining: 7.03s
10:	learn: 0.5948873	total: 73.1ms	remaining: 6.58s
11:	learn: 0.5881826	total: 75.3ms	remaining: 6.2s
12:	learn: 0.5823280	total: 77.4ms	remaining: 5.88s
13:	learn: 0.5764408	total: 79.6ms	remaining: 5.6s
14:	learn: 0.5694969	total: 81.7ms	remaining: 5.37s
15:	learn: 0.5628785	total: 83.9ms	remaining: 5.16s
16:	learn: 0.5562844	total: 86ms	remaining: 4.97s
17:	learn: 0.5502864	total: 88.3ms	remaining: 4.82s
18:	learn: 0.5449979	total: 90.4ms	remaining: 4.67s
19:	learn: 0.5400884	total: 92.6

GridSearchCV(cv=5, error_score=nan,
             estimator=<catboost.core.CatBoostClassifier object at 0x7fe3d257d690>,
             iid='deprecated', n_jobs=-1,
             param_grid={'depth': [4, 5, 7],
                         'learning_rate': [0.01, 0.02, 0.03]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [31]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'depth': 7, 'learning_rate': 0.02}
0.6129122731606044


In [32]:
cbc = grid_cv
cbc_pred = cbc.predict(X_test)

In [33]:
f1_score(y_test, cbc_pred)

0.7524752475247524

#### 6. Perceptron

In [56]:
per = Perceptron()
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3],
}

In [57]:
grid_cv = GridSearchCV(per, param_grid, n_jobs=-1, cv=5, scoring="f1")

In [58]:
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Perceptron(alpha=0.0001, class_weight=None,
                                  early_stopping=False, eta0=1.0,
                                  fit_intercept=True, max_iter=1000,
                                  n_iter_no_change=5, n_jobs=None, penalty=None,
                                  random_state=0, shuffle=True, tol=0.001,
                                  validation_fraction=0.1, verbose=0,
                                  warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03,
                                   0.1, 0.3],
                         'penalty': ['l2', 'l1', 'elasticnet']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [59]:
per = grid_cv
per_pred = per.predict(X_test)

In [60]:
f1_score(y_test, per_pred)

0.5396825396825397

#### 7. Multi-Layer Perceptron 

In [47]:
mlp = MLPClassifier()
param_grid = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
}

In [48]:
grid_cv = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=5, scoring="f1")


In [49]:
grid_cv.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__activation', 'estimator__alpha', 'estimator__batch_size', 'estimator__beta_1', 'estimator__beta_2', 'estimator__early_stopping', 'estimator__epsilon', 'estimator__hidden_layer_sizes', 'estimator__learning_rate', 'estimator__learning_rate_init', 'estimator__max_fun', 'estimator__max_iter', 'estimator__momentum', 'estimator__n_iter_no_change', 'estimator__nesterovs_momentum', 'estimator__power_t', 'estimator__random_state', 'estimator__shuffle', 'estimator__solver', 'estimator__tol', 'estimator__validation_fraction', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [50]:
grid_cv.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'

In [54]:
mlp = grid_cv
mlp_pred = mlp.predict(X_test)

In [55]:
f1_score(y_test, mlp_pred)

0.5168539325842697